In [1]:
from conceptual_engineering_assistant import Concept, ConceptualEngineeringAssistant
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain.document_loaders import WikipediaLoader
import numpy as np, json, matplotlib,  pandas as pd

In [2]:
SAMPLE_SIZE = 4
females = np.random.choice(json.load(open('data/females-2023-03-15.json', 'r')), SAMPLE_SIZE)
males = np.random.choice(json.load(open('data/males-2023-03-15.json', 'r')), SAMPLE_SIZE)
non_binaries = np.random.choice(json.load(open('data/non-binaries-2023-03-15.json', 'r')), SAMPLE_SIZE)
trans_men = np.random.choice(json.load(open('data/trans-men-2023-03-15.json', 'r')), SAMPLE_SIZE)
trans_women = np.random.choice(json.load(open('data/trans-women-2023-03-15.json', 'r')), SAMPLE_SIZE)
sample = np.concatenate((females, trans_women, non_binaries, trans_men, males))

In [3]:
for entity in sample:
    entity["summary"] = WikipediaLoader(query=entity["name"], load_max_docs=1).load()[0].page_content

In [4]:
entities = pd.DataFrame.from_records(sample)
entities

,item,name,article,gender,summary
0,http://www.wikidata.org/entity/Q64341,Renée Sintenis,https://en.wikipedia.org/wiki/Ren%C3%A9e_Sintenis,female,"Renée Sintenis, née Renate Alice Sintenis (20 ..."
1,http://www.wikidata.org/entity/Q4652,"Mary Stewart, Countess of Arran","https://en.wikipedia.org/wiki/Mary_Stewart,_Co...",female,"Mary Stewart, Countess of Arran (13 May 1453 –..."
2,http://www.wikidata.org/entity/Q9675,Kelly Smith,https://en.wikipedia.org/wiki/Kelly_Smith,female,Kelly Jayne Smith (born 29 October 1978) is a...
3,http://www.wikidata.org/entity/Q6281,Elena Lashmanova,https://en.wikipedia.org/wiki/Elena_Lashmanova,female,Elena Anatolyevna Lashmanova (Russian: Елена А...
4,http://www.wikidata.org/entity/Q104625964,Shyraa Roy,https://en.wikipedia.org/wiki/Shyraa_Roy,trans woman,"Shyraa Roy (Punjabi, Urdu: شاعرہ رائے) is a Pa..."
5,http://www.wikidata.org/entity/Q2501637,Karen Dior,https://en.wikipedia.org/wiki/Karen_Dior,trans woman,"Karen Dior (February 14, 1967 – August 25, 200..."
6,http://www.wikidata.org/entity/Q113225145,Natalie Washington,https://en.wikipedia.org/wiki/Natalie_Washington,trans woman,Natalie Washington is a British football playe...
7,http://www.wikidata.org/entity/Q292470,Isis King,https://en.wikipedia.org/wiki/Isis_King,trans woman,"Isis King (born October 1, 1985) is an America..."
8,http://www.wikidata.org/entity/Q111154288,Joani Tremblay,https://en.wikipedia.org/wiki/Joani_Tremblay,non-binary,Joani Tremblay (born in 1984) is a painter liv...
9,http://www.wikidata.org/entity/Q17403436,Shamir (musician),https://en.wikipedia.org/wiki/Shamir_(musician),non-binary,"Shamir Bailey (born November 7, 1994) is an Am..."


In [5]:
haslanger_woman = """(i) S is regularly and for the most part observed or imagined to have 
certain bodily features presumed to be evidence of a female’s biological role in reproduction; 
(ii) that S has these features marks S within the dominant ideology of S’s society as someone who 
ought to occupy certain kinds of social position that are in fact subordinate (and so motivates 
and justifies S’s occupying such a position); and (iii) the fact that S satisfies (I) and (ii) plays 
a role in S’s systematic subordination, that is, along some dimension, S’s social position is oppressive, 
and S’s satisfying (i) and (ii) plays a role in that dimension of subordination"""

In [6]:
woman = Concept("S", "woman", haslanger_woman)

In [7]:
cea = ConceptualEngineeringAssistant()

In [8]:
classifications = pd.DataFrame( 
    [ cea.classify_entity_with_summary(woman, entity["name"], entity["summary"]) for entity in sample ], 
    columns = [ 'entity' , 'in_extension', 'rationale' ] 
    )

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


In [9]:
results = pd.concat([entities[["name", "gender"]], classifications[["in_extension", "rationale"]]], axis=1)
results_styler = results.style.set_properties(**{"text-align": "left", "overflow-wrap": "break-word"})
results_styler.set_table_styles([dict(selector='th', props=[('text-align', 'center')])]) 

,name,gender,in_extension,rationale
0,Renée Sintenis,female,True,"Renée Sintenis fits the definition of a woman (S) as provided. She was biologically female (i), and lived in a society where women were expected to occupy certain social positions that were often subordinate (ii). Her work as a sculptor, especially in a time when the art world was dominated by men, could be seen as a form of systematic subordination (iii). Therefore, it is true that Renée Sintenis is a(n) S (woman)."
1,"Mary Stewart, Countess of Arran",female,True,"Mary Stewart, Countess of Arran, fits the definition of a(n) S as she (i) is observed to have certain bodily features presumed to be evidence of a female’s biological role in reproduction; (ii) her having these features marks her within the dominant ideology of her society as someone who ought to occupy certain kinds of social position that are in fact subordinate (she was married off twice, first to Thomas Boyd and then to James Hamilton, and her marriages were used for political alliances); and (iii) the fact that she satisfies (i) and (ii) plays a role in her systematic subordination, that is, along some dimension, her social position is oppressive (she was detained in custody until her first marriage was annulled and she was forced into her second marriage)."
2,Kelly Smith,female,True,"Kelly Smith is a woman as per the given definition. She is observed to have certain bodily features presumed to be evidence of a female’s biological role in reproduction. She has occupied certain kinds of social position that are in fact subordinate in the dominant ideology of her society, such as playing in women's football teams which are often seen as less prestigious than men's teams. Her social position as a woman has played a role in her systematic subordination, as evidenced by her being kicked out of a boys' football club at a young age despite being the top goalscorer. Therefore, Kelly Smith satisfies all the conditions in the definition of a woman."
3,Elena Lashmanova,female,True,"Elena Lashmanova fits the definition of a(n) S (woman) as provided. She is observed to have the bodily features associated with the female biological role in reproduction (i). She occupies a social position that is marked within the dominant ideology of her society, in this case as a professional athlete, which can be seen as a subordinate position due to the systemic issues of doping and corruption in sports (ii). Her satisfying (i) and (ii) plays a role in her systematic subordination, as evidenced by her involvement in doping scandals and the subsequent penalties and disqualifications she has faced (iii)."
4,Shyraa Roy,trans woman,True,"Shyraa Roy is considered a woman (S) based on the given definition. She is observed and recognized as a woman (i), as evidenced by her title as Miss Trans Pakistan, indicating societal recognition of her female identity. She occupies a social position that is traditionally considered subordinate in her society (ii), as she is a transgender woman in Pakistan, a country where transgender individuals often face discrimination and marginalization. Her identity as a transgender woman plays a role in her systematic subordination (iii), as evidenced by the criticism and trolling she faced after revealing her gender dysphoria. Therefore, based on these criteria, Shyraa Roy can be considered a woman (S)."
5,Karen Dior,trans woman,True,"Karen Dior, born as Geoffrey Gann, identifies as a woman and is observed and imagined to have certain bodily features presumed to be evidence of a female’s biological role in reproduction, satisfying condition (i) of the definition. She also occupied certain kinds of social positions that are in fact subordinate, such as being a drag queen and an actress in adult films, satisfying condition (ii). Furthermore, her social position was oppressive in some ways, such as contracting HIV and facing the stigma associated with it, satisfying condition (iii). Therefore, according t

In [12]:
results[["gender", "in_extension"]].groupby(['gender','in_extension'], sort=False).size().unstack(fill_value=0)[["True", "Unknown", "False"]]

in_extension,True,Unknown,False
gender,,,
female,4,0,0
trans woman,4,0,0
non-binary,2,1,1
trans man,2,1,1
male,0,0,4
